**Author:
Bibek Prasad Gupta**

In [1]:
import pandas as pd
import numpy as np
import math
import io
from tqdm import tqdm
from datetime import datetime

tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir('/content/drive/MyDrive/CA683')

Mounted at /content/drive/


**Read the bitcoin dataset**

In [3]:
# data = pd.read_csv("bitstampUSD.csv", nrows=240000)
data = pd.read_csv("bitstampUSD.csv")

In [ ]:
data.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Checking the columns which have null values and dropping the null rows**

In [4]:
# data[data.columns.values] = data[data.columns.values].ffill()
columns_to_check_for_null = ['Close']
data.shape
data.dropna(subset=columns_to_check_for_null, inplace=True)
data.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
478,1325346600,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
547,1325350740,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
548,1325350800,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
1224,1325391360,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000


In [ ]:
data.shape

(3484305, 8)

**Convert column to date format**

In [5]:
def convert_to_date(x):
    try:
      return datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M')
    except:
        print(x)

data["date"] = data.progress_apply(lambda x:convert_to_date(x["Timestamp"]),axis=1)

100%|██████████| 3484305/3484305 [00:56<00:00, 61264.78it/s]


**COnvert to datetime datatype**

In [7]:
data['date']= pd.to_datetime(data['date'], errors= "coerce")
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3484305 entries, 0 to 4727776
Data columns (total 9 columns):
 #   Column             Dtype         
---  ------             -----         
 0   Timestamp          int64         
 1   Open               float64       
 2   High               float64       
 3   Low                float64       
 4   Close              float64       
 5   Volume_(BTC)       float64       
 6   Volume_(Currency)  float64       
 7   Weighted_Price     float64       
 8   date               datetime64[ns]
dtypes: datetime64[ns](1), float64(7), int64(1)
memory usage: 265.8 MB


**Filter the data past 2016**

In [8]:
# Filter rows
start_year = pd.to_datetime('2016-01-01')
data = data[data['date'] >= start_year]

**Create index for grouper and calculate close price by one hour frequency**

In [9]:
data['time'] = data['date']
data.index = data['time']

crypto_usd_grouped = data.groupby(pd.Grouper(key="time", freq="1h")).agg(
    closed_price_by_hour = ('Close', np.mean)
)

In [ ]:
crypto_usd_grouped.head()

,closed_price_by_hour
time,
2016-01-01 00:00:00,431.466842
2016-01-01 01:00:00,430.464194
2016-01-01 02:00:00,431.013462
2016-01-01 03:00:00,431.600811
2016-01-01 04:00:00,434.281633


**Method to check the upward and down trend**

In [10]:
def movement_classifier(x):
    try:
        if x >= 0 :
            return 1
        else:
            return 0
    except:
        print(x)

**Calculate the price trend**

In [11]:
# Drop null rows
crypto_usd_grouped.dropna(subset=['closed_price_by_hour'], inplace=True)
# Calculate the difference between previous row
crypto_usd_grouped["price_diff"] = crypto_usd_grouped["closed_price_by_hour"].diff()
# Calculate price movement
crypto_usd_grouped["movement"] = crypto_usd_grouped["price_diff"].progress_apply(lambda x:movement_classifier(x))
crypto_usd_grouped.head()

100%|██████████| 43821/43821 [00:00<00:00, 658750.36it/s]


,closed_price_by_hour,price_diff,movement
time,,,
2016-01-01 00:00:00,431.466842,NaN,0
2016-01-01 01:00:00,430.464194,-1.002649,0
2016-01-01 02:00:00,431.013462,0.549268,1
2016-01-01 03:00:00,431.600811,0.587349,1
2016-01-01 04:00:00,434.281633,2.680822,1


In [ ]:
crypto_usd_grouped.shape

(43821, 3)

**Save data for future use**

In [ ]:
crypto_usd_grouped.to_csv("bitcoin_hourly_processed_data.csv")

**Merge sentiment and bitcoin prices data**

In [13]:
# Run sentiment_preprocessing notebook to generate "twitter_sentiments_hourly_processed_data.csv" file
data_sentiments = pd.read_csv("twitter_sentiments_hourly_processed_data.csv")
data_sentiments["time"] = pd.to_datetime(data_sentiments["time"], errors="coerce")
data_bitcoin = pd.read_csv("bitcoin_hourly_processed_data.csv")
data_bitcoin["time"] = pd.to_datetime(data_bitcoin["time"], errors="coerce")
merged_data = pd.merge(data_bitcoin, data_sentiments, on='time')

**Save data for future use**

In [14]:
# Final features to train the models
merged_data.to_csv("bitcoin_and_sentiments_data.csv")
merged_data.columns

Index(['time', 'closed_price_by_hour', 'price_diff', 'movement',
       'Daily_Weight_mean_by_hour', 'Daily_Weight_count_by_hour',
       'blob_sent_mean_by_hour', 'subjectivity_mean_by_hour',
       'vader_sent_mean_by_hour', 'tweet_diff', 'blob_sent_mean_by_hour_diff',
       'vader_sent_mean_by_hour_diff', 'tweet_movement', 'blob_sent_movement',
       'vader_sent_movement'],
      dtype='object')

In [ ]:
merged_data.head()

,time,closed_price_by_hour,price_diff,movement,Daily_Weight_mean_by_hour,Daily_Weight_count_by_hour,blob_sent_mean_by_hour,subjectivity_mean_by_hour,vader_sent_mean_by_hour,tweet_diff,blob_sent_mean_by_hour_diff,vader_sent_mean_by_hour_diff,tweet_movement,blob_sent_movement,vader_sent_movement
0,2016-02-19 08:00:00,418.409535,1.745839,1,0.0,1,0.0,0.15,-0.4019,0.0,-3.93183,-0.442529,1,0,0
1,2016-03-28 03:00:00,423.431613,0.030249,1,0.0,2,0.0,0.00,0.0000,1.0,0.00000,0.401900,1,1,1
2,2016-03-28 06:00:00,423.297407,-0.787831,0,0.0,1,0.0,0.00,0.0000,-1.0,0.00000,0.000000,0,1,1
3,2016-05-03 03:00:00,442.347273,0.348523,1,0.0,1,0.0,0.00,0.0000,0.0,0.00000,0.000000,1,1,1
4,2016-05-03 07:00:00,443.669063,0.486729,1,0.0,1,0.0,0.00,0.0000,0.0,0.00000,0.000000,1,1,1
